# WSOL - Backbone Network
---

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
if '..' not in sys.path: sys.path.append("..")

In [ ]:
import time
from src.classifier.agent import Agent
from src.utils.utils import create_directory
from src.utils.decorator import close_on_finish_decorator

In [ ]:
import torch
def run_me(name,i_limit=5,base_config='base'):
    torch.cuda.empty_cache()
    agent = Agent(name, base_config=base_config)
    logged_metrics = []
    
    agent.load_model() 
    agent.load_trainer()
    print(f"{agent.trainer.logger.log_dir.rsplit('/',1)[0]}/")
    create_directory(f"{agent.trainer.logger.log_dir.rsplit('/',1)[0]}/") 
    with open(f"{agent.trainer.logger.log_dir.rsplit('/',1)[0]}/hparams.json", "w") as write_file:
        json.dump(agent._config, write_file, indent = 6)
            
    try:
        i = 0
        while True:
            agent.load_model() 
            agent.load_trainer()
            print(f"Dataloader fold: {agent.dataloader.kfold_index}")
            close_on_finish_decorator(agent.trainer.fit, agent.trainer.logger.log_dir, agent.model, datamodule=agent.dataloader, message=agent._config) 
            
            with open(agent.trainer.logger.log_dir.rsplit("/",1)[0] + "/logged_metrics", 'a+') as f:
                f.write(str(agent.trainer.logged_metrics))
                
            logged_metrics.append(agent.trainer.logged_metrics)
            time.sleep(10)
            i += 1
            #del tr

            
            if not agent.dataloader.next_fold() or i == i_limit and i_limit != -1: break
            torch.cuda.empty_cache()
    except Exception as e:
        torch.cuda.empty_cache()
        raise e
    print(f"{name}: {logged_metrics}")

In [ ]:
models = ["customCNN", "resnet18_brew2","vgg11_brew","vgg16","resnet50"]
for name in models:
    run_me(name, base_config='base',i_limit=-1)

In [ ]:
models = ["customCNN", "resnet18_brew2","vgg11_brew","vgg16","resnet50"]
for name in models:
    run_me(name, base_config='augmented',i_limit=-1)

In [ ]:
models = ["customCNN", "resnet18_brew2","vgg11_brew","vgg16","resnet50"]
for name in models:
    run_me(name, base_config='no_augmented',i_limit=-1)